# UTS Course Data Script
The goal of this python script is to scrape the UTS handbook for degree, course and subject data. This is so that it can be stored in our database and the necessary algorithms can be run on it, so that we can recommend subjects automatically.

## Library Imports
* requests library: Uses HTTP to request the webpage from a server with a given URL (in this case https://www.handbook.uts.edu.au/).
* BeautifulSoup library: Parsers HTML files so that it can be searched and filtered.
* json library: Reading, writing and handling JSON files.

In [1]:
import requests
from bs4 import BeautifulSoup
import json

In [2]:
baseUrl = 'https://www.handbook.uts.edu.au/'

In [3]:
def getPageHTML(url):
    return BeautifulSoup(requests.get(url).content, 'html.parser')

In [51]:
def getDegrees(courseArea, graduateType):
    degrees = []
    degreesPage = BeautifulSoup(requests.get(f"{baseUrl}{courseArea}/{graduateType}/index.html").content, 'html.parser')
    tables = degreesPage.find(id='content').find(class_='ie-images').find_all('table')
    
    if len(tables) <= 1:
        return None
    else:
        for table in tables:
            tableCells = table.find_all("td")
            for cellNumber in range(1, len(tableCells)):
                isDegreeCode = tableCells[cellNumber-1].text.strip()[0] == "C"
                degrees.append({
                    'name': tableCells[cellNumber if isDegreeCode else cellNumber-1].text.strip(),
                    'degreeCode': (tableCells[cellNumber-1 if isDegreeCode else cellNumber].text).lower().strip()
                })
                
    return degrees

In [52]:
indexPage = requests.get(baseUrl)

soup = BeautifulSoup(indexPage.content, 'html.parser')
# print(soup.find(class_='toc').find_all('a', href=True)[0]['href'])

# links to the pages for undergraduate and post graduate degrees in all course areas
courseAreas = {}

for courseArea in soup.find(class_='toc').find_all('a', href=True):
    abbreviation = courseArea['href'][1:4]
    
    courseAreas[courseArea.text] = {
        'abbreviation': abbreviation,
        'url': f"https://www.handbook.uts.edu.au{courseArea['href']}",
        'ug_degrees': getDegrees(abbreviation, 'ug'),
        'pg_degrees': getDegrees(abbreviation, 'pg')
    }

print(courseAreas)

{'Analytics and Data Science': {'abbreviation': 'ads', 'url': 'https://www.handbook.uts.edu.au/ads/index.html', 'ug_degrees': None, 'pg_degrees': [{'name': 'Doctor of Philosophy', 'degreeCode': 'c02062'}, {'name': 'Doctor of Philosophy', 'degreeCode': 'c03064'}, {'name': 'Master of Learning Analytics (Research)', 'degreeCode': 'c03064'}, {'name': 'Master of Data Science and Innovation', 'degreeCode': 'c04372'}, {'name': 'Master of Data Science and Innovation', 'degreeCode': 'c06124'}, {'name': 'Graduate Diploma in Data Science and Innovation', 'degreeCode': 'c06124'}, {'name': 'Graduate Diploma in Data Science and Innovation', 'degreeCode': 'c11274'}, {'name': 'Graduate Certificate in Data Science and Innovation', 'degreeCode': 'c11274'}]}, 'Business': {'abbreviation': 'bus', 'url': 'https://www.handbook.uts.edu.au/bus/index.html', 'ug_degrees': [{'name': 'Bachelor of Business', 'degreeCode': 'c10026'}, {'name': 'Bachelor of Business', 'degreeCode': 'c10226'}, {'name': 'Bachelor of Bus

## Scraping subject data seperately
Since many degrees share common subjects, I will be scraping the subject data seperately so I can normalise the json. Which in turn will reduce the data redundancy of the database.

In [55]:
subjects = {}

for key in courseAreas:
    courseArea = courseAreas[key]
    if courseArea['ug_degrees'] != None:
        for degree in courseArea['ug_degrees']:
            tables = getPageHTML(f"{baseUrl}courses/{degree['degreeCode']}.html").find_all('table')
            for tableCell in tables[len(tables) - 1]:
                print(tableCell)



<tr><td colspan="3"><strong>Accounting and Finance majors, Autumn commencing, full time</strong></td></tr>


<tr><td colspan="3"><strong>Year 1</strong></td></tr>


<tr><td colspan="3">
<em>
											Autumn session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/26100.html">26100</a> Integrating Business Perspectives</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/22107.html">22107</a> Accounting for Business Decisions A</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/23115.html">23115</a> Economics for Business</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/26134.html">26134</a> Business Statistics</td>
<td> </td>
<td>6cp</td></tr>


<tr><td colspan="3">
<em>
											Spring session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/22207.html">22207</a> Acc



<tr><td><a href="https://handbook.uts.edu.au/directory/stm90850.html">STM90850</a> Core subjects (Business)</td><td> </td><td>48cp</td></tr>


<tr><td><a href="https://handbook.uts.edu.au/directory/cbk90812.html">CBK90812</a> Major choice (Business)</td><td> </td><td>48cp</td></tr>


<tr><td><a href="https://handbook.uts.edu.au/directory/cbk90813.html">CBK90813</a> Year 3 (Business)</td><td> </td><td>48cp</td></tr>


<tr><td class="right">Total</td><td> </td><td>144cp</td></tr>




<tr><td colspan="3"><strong></strong></td></tr>


<tr><td colspan="3"><strong>Year 1</strong></td></tr>


<tr><td colspan="3">
<em>
											Autumn session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/22107.html">22107</a> Accounting for Business Decisions A</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/22207.html">22207</a> Accounting for Business Decisions B</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a h


<tr><td colspan="3"><strong></strong></td></tr>


<tr><td colspan="3"><strong>Year 1</strong></td></tr>


<tr><td colspan="3">
<em>
											Autumn session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/22107.html">22107</a> Accounting for Business Decisions A</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/22207.html">22207</a> Accounting for Business Decisions B</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/22605.html">22605</a> Accounting Information Systems</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/25300.html">25300</a> Fundamentals of Business Finance</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/23115.html">23115</a> Economics for Business</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/26100.html">26100</a



<tr><td colspan="3"><strong>Typical block mode program, full time</strong></td></tr>


<tr><td colspan="3"><strong>Year 1</strong></td></tr>


<tr><td colspan="3">
<em>
											Autumn session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/21597.html">21597</a> Principles of Money and Finance</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/21393.html">21393</a> The Business Environment</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/21493.html">21493</a> Principles of Marketing</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/21063.html">21063</a> Contemporary Issues in the Indigenous Context (Project)</td>
<td> </td>
<td>6cp</td></tr>


<tr><td colspan="3">
<em>
											Spring session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/21067.html">21067</a> M



<tr><td colspan="3"><strong>Events major, Autumn commencing, full time</strong></td></tr>


<tr><td colspan="3"><strong>Year 1</strong></td></tr>


<tr><td colspan="3">
<em>
											Autumn session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/21637.html">21637</a> Event and Entertainment Contexts</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/21639.html">21639</a> Event Impacts and Legacies</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/24108.html">24108</a> Marketing Foundations</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/21129.html">21129</a> Managing People and Organisations</td>
<td> </td>
<td>6cp</td></tr>


<tr><td colspan="3">
<em>
											Spring session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/21640.html">21640</a> Event Management<

...



<tr><td colspan="3"><strong></strong></td></tr>


<tr><td colspan="3"><strong>Year 1</strong></td></tr>


<tr><td colspan="3">
<em>
											Autumn session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/84610.html">84610</a> Inside Design</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/84111.html">84111</a> Understanding Three-dimensional Form</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/84116.html">84116</a> Product Design Communication A</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/85502.html">85502</a> Researching Design Histories</td>
<td> </td>
<td>6cp</td></tr>


<tr><td colspan="3">
<em>
											Spring session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/84611.html">84611</a> Integrated Product Design</td>
<td> </td>
<td>6cp</td></tr>


<t



<tr><td colspan="3"><strong></strong></td></tr>


<tr><td colspan="3"><strong>Year 1</strong></td></tr>


<tr><td colspan="3">
<em>
											Autumn session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/83119.html">83119</a> Thinking Fashion</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/83621.html">83621</a> Studio: Foundations in Patternmaking and Construction 1</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/83622.html">83622</a> Studio: Fashion Illustration Fundamentals 1</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/85503.html">85503</a> Thinking Through Design</td>
<td> </td>
<td>6cp</td></tr>


<tr><td colspan="3">
<em>
											Spring session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/83231.html">83231</a> Fashion Cultures</td>
<td> </td>
<



<tr><td colspan="3"><strong></strong></td></tr>


<tr><td colspan="3"><strong>Year 1</strong></td></tr>


<tr><td colspan="3">
<em>
											Autumn session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/87631.html">87631</a> VC Design Studio: The Politics of Image and Text</td>
<td> </td>
<td>12cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/87100.html">87100</a> VC Design Theory: Critical Approaches to Visual Culture</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/85502.html">85502</a> Researching Design Histories</td>
<td> </td>
<td>6cp</td></tr>


<tr><td colspan="3">
<em>
											Spring session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/87632.html">87632</a> VC Design Studio: The Ethics of Image and Text</td>
<td> </td>
<td>12cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/87222.html">87



<tr><td colspan="3"><strong></strong></td></tr>


<tr><td colspan="3"><strong>Year 1</strong></td></tr>


<tr><td colspan="3">
<em>
											Autumn session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/16467.html">16467</a> Built Environment Law and Ethics</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/16466.html">16466</a> Built Environment Economics</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/16631.html">16631</a> Construction and Development Process</td>
<td> </td>
<td>6cp</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/16234.html">16234</a> Property Valuation</td>
<td> </td>
<td>6cp</td></tr>


<tr><td colspan="3">
<em>
											Spring session
												
										</em>
</td></tr>


<tr>
<td><a href="https://handbook.uts.edu.au/subjects/16632.html">16632</a> Built Environment Law and Professional Practice</td>
<td> 

IndexError: list index out of range

### Degree Data is written to JSON file

In [36]:
with open('courseData.json', 'w') as fp:
    json.dump(courseAreas, fp, indent = 4)